I got a question about interpreting plots you get from [Arviz](https://arviz-devs.github.io/). This particular one was about LOO-PIT which you can find [here](https://arviz-devs.github.io/arviz/api/generated/arviz.plot_loo_pit.html).

Let's go into more details about why you ever need this plot, how to work with it and which actions to take after you see it. A case study.

In [3]:
import pymc as pm
import arviz as az

## A generative model

First of all, we need a model to demonstrate the pathologies that can be diagnosed with the LOO-PIT. I will use linear regression with robust likelihood to show all possible cases that can be fixed just by looking at the plot.

But wait, what is LOO-PIT is you ask? Ok, let's give some theory before we start.

* LOO stays for Leave One Out validation. We usually want LOO to cross-validate our model. We train it on all but the one observarions and then make predictions on the one that was holdout. We do that for every single observation.
* PIT stays for Probability Integral Transformation. Briefly, it transforms any continuous distribution to the Uniform distribution.

While it is relatively clear what is LOO, it is not super clear, what is PIT. Let's make some visualizations.

fdf